In [ ]:
from telethon.sync import TelegramClient

# Replace these with your own values
api_id = '22908602'
api_hash = '20bdd2238cc3dbb7014870281cce444e'
phone_number = '+251701425988'

# Create the client and connect
client = TelegramClient('session_name1', api_id, api_hash)

async def connect_telegram():
    await client.start(phone_number)
    print("Connected to Telegram")

await connect_telegram()


Signed in successfully as C48; remember to not break the ToS or you will risk an account ban!
Connected to Telegram


Server closed the connection: [WinError 10054] An existing connection was forcibly closed by the remote host
Attempt 1 at connecting failed: OSError: [Errno 10051] Connect call failed ('149.154.167.91', 443)
Attempt 2 at connecting failed: OSError: [Errno 10051] Connect call failed ('149.154.167.91', 443)
Attempt 3 at connecting failed: OSError: [Errno 10051] Connect call failed ('149.154.167.91', 443)
Attempt 4 at connecting failed: OSError: [Errno 10051] Connect call failed ('149.154.167.91', 443)
Attempt 5 at connecting failed: OSError: [Errno 10051] Connect call failed ('149.154.167.91', 443)
Attempt 6 at connecting failed: OSError: [Errno 10051] Connect call failed ('149.154.167.91', 443)
Attempt 1 at connecting failed: OSError: [Errno 10051] Connect call failed ('149.154.167.91', 443)


In [ ]:
import time
import pandas as pd
from telethon.tl.functions.messages import GetHistoryRequest
from telethon.tl.types import PeerChannel

import os

from telethon import TelegramClient
from telethon.tl.functions.messages import GetHistoryRequest
from telethon.tl.types import MessageMediaPhoto

message scrapping 

In [8]:

async def scrape_channel_messages(channel_username, max_messages=10000, batch_size=100, sleep_time=2):
    # Get the channel entity
    channel = await client.get_entity(channel_username)

    # Variables for scraping
    offset_id = 0
    all_messages = []

    # Extract channel details
    channel_title = channel.title if hasattr(channel, 'title') else 'N/A'
    channel_id = channel.id

    while len(all_messages) < max_messages:
        print(f"Scraping messages... Collected {len(all_messages)} so far.")

        try:
            # Fetch the message history
            history = await client(GetHistoryRequest(
                peer=channel,
                offset_id=offset_id,
                offset_date=None,
                add_offset=0,
                limit=batch_size,
                max_id=0,
                min_id=0,
                hash=0
            ))

            # Break the loop if no more messages are returned
            if not history.messages:
                break

            # Add messages to the list
            messages = history.messages
            all_messages.extend(messages)

            # Update the offset_id to the last message's ID to avoid duplicates
            offset_id = messages[-1].id

            # Sleep to respect Telegram's rate limit
            time.sleep(sleep_time)

        except Exception as e:
            print(f"An error occurred while fetching messages: {e}")
            break  # Break the loop if there's an error fetching messages

    # Limit the number of messages to max_messages
    all_messages = all_messages[:max_messages]

    # Prepare the data for CSV export
    data = []
    for msg in all_messages:
        try:
            if msg.message:  # Only save non-empty messages
                media_path = ''
                if msg.media:  # Check if there's media
                    try:
                        # Handle different types of media
                        if hasattr(msg.media, 'document'):
                            media_path = msg.media.document.attributes[0].file_name
                        elif hasattr(msg.media, 'photo'):
                            media_path = 'Photo media'
                        elif hasattr(msg.media, 'video'):
                            media_path = 'Video media'
                        else:
                            media_path = 'Other media'
                    except Exception as media_error:
                        print(f"Error processing media: {media_error}")
                        media_path = 'Error processing media'

                data.append({
                    'Channel Title': channel_title,
                    'Channel Username': channel_username,
                    'ID': msg.id,
                    'Message': msg.message,
                    'Date': msg.date,
                    'Media Path': media_path
                })

        except Exception as msg_error:
            print(f"Error processing message ID {msg.id}: {msg_error}")

    # Convert to DataFrame and save to CSV
    df = pd.DataFrame(data)
    df.to_csv(r'C:\Users\Yibabe\Desktop\10academykifiyaAIMweek-7\data\scraped_data.csv', index=False)
    print("Scraping complete. Data saved to 'scraped_data.csv'.")

# Scrape up to 10,000 messages and save to CSV
await scrape_channel_messages('@yetenaweg', max_messages=10000, batch_size=100, sleep_time=0.2)


Scraping messages... Collected 0 so far.
Scraping messages... Collected 100 so far.
Scraping messages... Collected 200 so far.
Scraping messages... Collected 300 so far.
Scraping messages... Collected 400 so far.
Scraping messages... Collected 500 so far.
Scraping messages... Collected 600 so far.
Scraping messages... Collected 700 so far.
Scraping messages... Collected 800 so far.
Scraping messages... Collected 900 so far.
Scraping messages... Collected 1000 so far.
Scraping messages... Collected 1079 so far.
Error processing media: 'DocumentAttributeVideo' object has no attribute 'file_name'
Error processing media: 'DocumentAttributeAudio' object has no attribute 'file_name'
Error processing media: 'DocumentAttributeAudio' object has no attribute 'file_name'
Error processing media: 'DocumentAttributeAudio' object has no attribute 'file_name'
Error processing media: 'DocumentAttributeAudio' object has no attribute 'file_name'
Error processing media: 'DocumentAttributeAudio' object has

image scraping

In [5]:
async def scrape_image_messages(channel_username, max_images=10000, batch_size=100, sleep_time=2):
    # Get the channel entity
    channel = await client.get_entity(channel_username)
    
    # Variables for scraping
    offset_id = 0
    all_images = []

    while len(all_images) < max_images:
        print(f"Scraping images... Collected {len(all_images)} so far.")

        # Fetch the message history
        history = await client(GetHistoryRequest(
            peer=channel,
            offset_id=offset_id,
            offset_date=None,
            add_offset=0,
            limit=batch_size,
            max_id=0,
            min_id=0,
            hash=0
        ))

        # Break the loop if no more messages are returned
        if not history.messages:
            break

        # Add image messages to the list
        messages = history.messages
        for msg in messages:
            if isinstance(msg.media, MessageMediaPhoto):  # Check for photo media
                image_url = f"https://t.me/c/{channel.id}/{msg.id}"  # Construct the URL to the image
                all_images.append({
                    'ID': msg.id,
                    'Date': msg.date,
                    'Image URL': image_url
                })

        # Update the offset_id to the last message's ID to avoid duplicates
        offset_id = messages[-1].id

        # Sleep to respect Telegram's rate limit
        time.sleep(sleep_time)

    # Limit the number of images to max_images
    all_images = all_images[:max_images]

    # Convert to DataFrame and save to CSV
    df = pd.DataFrame(all_images)
    df.to_csv(r'C:\Users\Yibabe\Desktop\10academykifiyaAIMweek-7\data\image_messages.csv', index=False)
    print("Scraping complete. Image data saved to 'image_messages.csv'.")

# Run the scraping function
await scrape_image_messages('@CheMed123', max_images=10000, batch_size=100, sleep_time=0.2)


Scraping images... Collected 0 so far.
Scraping images... Collected 67 so far.
Scraping complete. Image data saved to 'image_messages.csv'.


we scrap the telegram channels to get the url of the image

In [8]:


# Directory to save images
output_directory = r'C:\Users\Yibabe\Desktop\10academykifiyaAIMweek-7\data\images1'
os.makedirs(output_directory, exist_ok=True)  # Create the directory if it doesn't exist

async def scrape_image_messages(channel_username, max_images=10000, batch_size=100, sleep_time=2):
    # Get the channel entity
    channel = await client.get_entity(channel_username)
    
    # Variables for scraping
    offset_id = 0
    all_images = []

    while len(all_images) < max_images:
        print(f"Scraping images... Collected {len(all_images)} so far.")

        # Fetch the message history
        history = await client(GetHistoryRequest(
            peer=channel,
            offset_id=offset_id,
            offset_date=None,
            add_offset=0,
            limit=batch_size,
            max_id=0,
            min_id=0,
            hash=0
        ))

        # Break the loop if no more messages are returned
        if not history.messages:
            break

        # Add image messages to the list
        messages = history.messages
        for msg in messages:
            if isinstance(msg.media, MessageMediaPhoto):
                # Download the image to the specified directory
                image_file = await client.download_media(msg, output_directory)
                
                if image_file:  # If the image is successfully downloaded
                    all_images.append({
                        'ID': msg.id,
                        'Date': msg.date,
                        'Image Path': image_file
                    })

        # Update the offset_id to the last message's ID to avoid duplicates
        offset_id = messages[-1].id

        # Sleep to respect Telegram's rate limit
        time.sleep(sleep_time)

    # Limit the number of images to max_images
    all_images = all_images[:max_images]

    # Convert to DataFrame and save to CSV
    df = pd.DataFrame(all_images)
    df.to_csv(r'C:\Users\Yibabe\Desktop\10academykifiyaAIMweek-7\data\image1.csv', index=False)
    print("Scraping complete. Image data saved to 'image_messages.csv'.")

# Main function to run the scraping
async def main():
    await client.start()  # Start the client
    await scrape_image_messages('@CheMed123', max_images=10000, batch_size=100, sleep_time=0.2)

# Run the main function
await main()



Scraping images... Collected 0 so far.
Scraping images... Collected 67 so far.
Scraping complete. Image data saved to 'image_messages.csv'.
